<a href="https://colab.research.google.com/github/amoakoh22/sa_ghana_oil_boom_or_bust/blob/main/ghana_oil_boom_or_bust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Environment Setup
!pip install pmdarima
!pip install --upgrade statsmodels

# Mount Google Drive
from google.colab import drive


# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose


# Data Loading & Preprocessing
# Load Ghana oil data (assumes CSV format)
oil_data = pd.read_csv('/content/drive/MyDrive/ghana_oil_data.csv',
                      parse_dates=['Date'],
                      index_col='Date')

# Handle missing values
oil_data.interpolate(method='time', inplace=True)

# Resample to monthly frequency
monthly_data = oil_data.resample('M').mean()

# Plot original data
plt.figure(figsize=(12,6))
plt.plot(monthly_data)
plt.title('Ghana Crude Oil Production/Prices Over Time')
plt.xlabel('Year')
plt.ylabel('Barrels per Day / Price (USD)')
plt.grid(True)
plt.show()

# Time Series Decomposition
decomposition = seasonal_decompose(monthly_data, model='additive')
decomposition.plot()
plt.suptitle('Time Series Decomposition')
plt.tight_layout()
plt.show()


# Stationarity Check
def adf_test(series):
    from statsmodels.tsa.stattools import adfuller
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')

print("Augmented Dickey-Fuller Test:")
adf_test(monthly_data)


# ARIMA Model Implementation
# Auto ARIMA for parameter selection
auto_model = auto_arima(monthly_data, seasonal=False, trace=True,
                        error_action='ignore', suppress_warnings=True,
                        stepwise=True)

# Best parameters
arima_order = auto_model.order
print(f"Best ARIMA Order: {arima_order}")

# Train ARIMA model
arima_model = ARIMA(monthly_data, order=arima_order)
arima_fit = arima_model.fit()

# Forecast
arima_forecast = arima_fit.forecast(steps=12)
plt.figure(figsize=(12,6))
plt.plot(monthly_data, label='Historical')
plt.plot(arima_forecast, label='ARIMA Forecast')
plt.title('12-Month ARIMA Forecast')
plt.legend()
plt.show()

# SARIMA Model Implementation
# Seasonal decomposition
auto_seasonal_model = auto_arima(monthly_data, seasonal=True, m=12,
                                trace=True, error_action='ignore',
                                suppress_warnings=True, stepwise=True)

# Best parameters
sarima_order = auto_seasonal_model.order
seasonal_order = auto_seasonal_model.seasonal_order
print(f"SARIMA Order: {sarima_order}x{seasonal_order}")

# Train SARIMA model
sarima_model = SARIMAX(monthly_data,
                      order=sarima_order,
                      seasonal_order=seasonal_order)
sarima_fit = sarima_model.fit(disp=False)

# Forecast
sarima_forecast = sarima_fit.get_forecast(steps=12)
conf_int = sarima_forecast.conf_int()
plt.figure(figsize=(12,6))
plt.plot(monthly_data, label='Historical')
plt.plot(sarima_forecast.predicted_mean, label='SARIMA Forecast')
plt.fill_between(conf_int.index,
                conf_int.iloc[:, 0],
                conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.title('12-Month SARIMA Forecast with Confidence Interval')
plt.legend()
plt.show()


# Model Evaluation
def evaluate_model(model, name):
    # AIC/BIC
    aic = model.aic
    bic = model.bic
    # RMSE
    predictions = model.predict(start=1, end=len(monthly_data))
    rmse = np.sqrt(mean_squared_error(monthly_data, predictions))

    print(f"{name} Evaluation:")
    print(f"AIC: {aic:.2f}")
    print(f"BIC: {bic:.2f}")
    print(f"RMSE: {rmse:.2f}\n")

evaluate_model(arima_fit, 'ARIMA')
evaluate_model(sarima_fit, 'SARIMA')